In [ ]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

--2021-02-23 18:10:30--  https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl
Resolving docs.google.com (docs.google.com)... 172.217.204.101, 172.217.204.113, 172.217.204.139, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5bo4u8jbttgn5ar967junsb9mtp2h6qa/1614103800000/07828702499945486090/*/1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl?e=download [following]
--2021-02-23 18:10:31--  https://doc-0s-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5bo4u8jbttgn5ar967junsb9mtp2h6qa/1614103800000/07828702499945486090/*/1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl?e=download
Resolving doc-0s-5k-docs.googleusercontent.com (doc-0s-5k-docs.googleusercontent.com)... 142.250.98.132, 2607:f8b0:400c:c1a::84
Connecting to doc-0s-5k-docs.googleusercontent.com (doc-0s-

In [ ]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

     |████████████████████████████████| 7.0MB 5.1MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import numpy as np
import pandas as pd

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/ronitf/heart-disease-uci). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

   Найдите долю правильных ответов полученного алгоритма на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=None, random_state=13)
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=13, splitter='best')

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, dt.predict(X_train)) * 100

100.0

2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Заметно ли переобучение?

In [ ]:
accuracy_score(y_test, dt.predict(X_test)) * 100

67.10526315789474

3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`. 
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    Какое значение глубины дерева получилось оптимальным?

In [ ]:
from sklearn.model_selection import GridSearchCV

dt1 = DecisionTreeClassifier(random_state=13)
param_grid = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
              'max_features': ['auto', 'log2', None],
              'min_samples_leaf': range(1, 10),
              'min_samples_split': range(2, 10),
              'criterion': ['gini', 'entropy']}
clf = GridSearchCV(estimator=dt1, param_grid=param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)
clf.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 9}

4. Какое лучшее усредненное значение доли правильных ответов получилось на кросс-валидации (для оптимальных значений гиперпараметров)? Вам поможет атрибут `best_score_`. Ответ округлите до двух знаков после запятой и дайте в процентах.

In [ ]:
clf.best_score_ * 100

83.23671497584542

5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

In [ ]:
dt = DecisionTreeClassifier(random_state=13, criterion='gini', max_depth=9, max_features='auto',
                            min_samples_leaf=3, min_samples_split=9)
dt.fit(X_train, y_train)
print(accuracy_score(y_train, dt.predict(X_train)) * 100)


90.7488986784141


6. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Уменьшилось ли переобучение?

In [ ]:
print(accuracy_score(y_test, dt.predict(X_test)) * 100)

73.68421052631578


7. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC. Ответ округлите до двух знаков после запятой.

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test, dt.predict_proba(X_test)[:,1]))

0.7716701902748413


8. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [ ]:
dt.feature_importances_

array([0.00768745, 0.11857961, 0.        , 0.00622126, 0.03031571,
       0.        , 0.00554163, 0.15552678, 0.15383366, 0.06550299,
       0.23535585, 0.18479243, 0.03664263])

In [ ]:
np.argmax(dt.feature_importances_)

10

In [ ]:
data.columns[10]

'slope'